# Camada Gold – Modelagem Analítica e Respostas de Negócio

## Objetivo
Construir tabelas analíticas e responder às perguntas de negócio
definidas no escopo do MVP:

1) Tempo médio entre lançamento e entrada no catálogo (filmes)
2) Evolução da diversidade do catálogo ao longo do tempo (gêneros únicos por ano de adição)
3) Quais gêneros são mais comuns na Disney+



## Pergunta 1
Qual é o tempo médio entre o lançamento e a entrada no catálogo Disney+ para filmes?


## Entradas (Silver)
- `silver.disney_titles_clean`: base canônica
- `silver.title_genres`: tabela normalizada com gêneros por título (show_id, genre)

## Saídas (Gold)
Dimensões e ponte:
- `gold.dim_title`
- `gold.dim_genre`
- `gold.bridge_title_genre`

Agregações analíticas:
- `gold.agg_time_to_catalog_movies`
- `gold.agg_genre_diversity_by_year_added`
- `gold.agg_genre_counts`


In [0]:
#Criando 'gold.dim_title'

from pyspark.sql import functions as F

titles = (spark.table("silver.disney_titles_clean")
          .select(
              "show_id",
              "type",
              "title",
              F.col("release_year_int").alias("release_year"),
              "rating",
              "date_added_dt",
              "duration_minutes",
              "seasons"
          ))

(titles.write.format("delta")
 .mode("overwrite")
 .saveAsTable("gold.dim_title"))


In [0]:

#Criar 'gold.dim_genre' e 'gold.bridge_title_genre'

genres = (spark.table("silver.title_genres")
          .select("show_id", F.trim(F.col("genre")).alias("genre"))
          .filter(F.col("genre").isNotNull() & (F.col("genre") != "")))

dim_genre = (genres.select("genre").dropDuplicates()
             .withColumn("genre_id", F.sha2(F.col("genre"), 256)))

bridge = (genres.join(dim_genre, on="genre", how="inner")
          .select("show_id", "genre_id"))

(dim_genre.write.format("delta").mode("overwrite").saveAsTable("gold.dim_genre"))
(bridge.write.format("delta").mode("overwrite").saveAsTable("gold.bridge_title_genre"))


In [0]:
%sql
SELECT
  AVG(YEAR(date_added_dt) - release_year) AS tempo_medio_anos
FROM silver.disney_titles_clean
WHERE type = 'Movie' AND date_added_dt IS NOT NULL;


## Pergunta 2
O catálogo está ficando mais diverso ao longo do tempo?

Métrica utilizada:
- Quantidade de gêneros únicos por ano de adição


In [0]:
df = spark.table("silver.disney_titles_clean").filter("date_added_dt IS NOT NULL")

df_genres = (
    df
    .withColumn("genre", F.explode(F.split("listed_in", ", ")))
    .withColumn("year_added", F.year("date_added_dt"))
)

df_genres.write.format("delta").mode("overwrite").saveAsTable("gold.genre_diversity")


In [0]:
%sql
SELECT
  year_added,
  COUNT(DISTINCT genre) AS unique_genres
FROM silver.title_genres
GROUP BY year_added
ORDER BY year_added;


In [0]:
pdf = spark.sql("""
SELECT year_added, COUNT(DISTINCT genre) AS unique_genres
FROM silver.title_genres
GROUP BY year_added
ORDER BY year_added
""").toPandas()

ax = pdf.plot(x="year_added", y="unique_genres", kind="line", title="Diversidade (gêneros únicos) por ano de adição")
ax.set_xlabel("Ano")
ax.set_ylabel("Gêneros únicos")


## Pergunta 3
Identificação de perfis de conteúdo utilizando atributos categóricos
(type, rating e gêneros).
